In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from collections import defaultdict
from torch.autograd import Variable
import torch.optim as optim
%matplotlib inline

In [ ]:
train_foldername = 'en-valid-10k'
train_filename = 'qa1_train.txt'
train_fname = './bAbI_Data/'+str(train_foldername)+'/'+str(train_filename)

valid_foldername = 'en-valid-10k'
valid_filename = 'qa1_valid.txt'
valid_fname = './bAbI_Data/'+str(valid_foldername)+'/'+str(valid_filename)

In [ ]:
train_dat_aux = []
valid_dat_aux = []
punctuations = ['.',',','?']

for l in open(train_fname):
    temp = ''.join(ch for ch in l if ch not in punctuations)
    train_dat_aux.append(temp.strip().split())
    
for l in open(valid_fname):
    temp = ''.join(ch for ch in l if ch not in punctuations)
    valid_dat_aux.append(temp.strip().split())

print(len(train_dat_aux))
print(len(valid_dat_aux))

In [ ]:
def hasDigits(input_str):
    return any(char.isdigit() for char in input_str)

In [ ]:
def create_vocab(data,unk_thres=0):
    aux = defaultdict(int)
    for i in range(len(data)):
        for j in range(1,len(data[i])):
            if hasDigits(data[i][j]):
                break
            aux[data[i][j]] += 1
    vocab = []
    unk_list = []
    for i in aux:
        if aux[i] < unk_thres:
            if not unk_list:
                vocab.append('UNK')
            unk_list.append(i)
        else:
            vocab.append(i)
    
    return vocab, unk_list

In [ ]:
vocab, unk_list = create_vocab(train_dat_aux,0)
vocab_size = len(vocab)

In [ ]:
def smax(x):
    y = np.divide(np.exp(x),np.sum(np.exp(x)))
    return ydef comp(out,target):
    if (target == np.argmax(smax(out))):
        return 1
    else:
        return 0

In [ ]:
def train(model,tr_dt,epochs=10,eta=0.0001):
    
    optimiser = optim.Adam(lr=eta)
    loss = nn.CrossEntropyLoss()
    tr_shape = tr_dt.shape
    eps = []
    l_tr = []
    accuracy = []
    for epoch in range(epochs):
        count=0;
        n_corr = 0;
        for i in range(tr_shape[0]):
            l_temp = 0
            tag = 'q'
            if(tr_dt[i,-1:]==-1):
                tag = 's'
                model(tr_dt[i,1:],tag)
            elif(tr_dt[i,--1:]==-2):
                tag = 'f'
                model(tr_dt[i,1:],tag)
            else:
                count+=1
                out = model(tr_dt[i,1:],tag)
                target = tr_dt[i,-1:]
                optimiser.zero_grad()

                loss_tr = loss(out,target)
                loss_tr.backward()
                optimser.step()
                l_temp += loss_tr.data[0]
                if i % 15 == 14:
                    print('[%d, %5d] loss: %.3f' % (epoch+1,i+1,l_temp/15))

                n_corr += comp(out,target)
        acc = n_corr/count*100
        l_tr.append(l_temp)
        eps.append(epoch)
        accuracy.append(acc)
    plt.plot(eps,l_tr)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(['Training Loss'])
    plt.savefig('Loss1.png')
    plt.show()

    plt.plot(eps,accuracy)
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy%')
    plt.legend(['Training Accuracy'],loc=4)
    plt.savefig('Acc2.png')
    plt.show()
    return l_tr, accuracy